In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import tqdm
import multiprocessing as mp
from config import *

result_directory = result_parent_directory + os.sep + 'step3_btms_sizing' + os.sep + 'analysis'
os.makedirs(result_directory, exist_ok=True)
figure_directory = result_parent_directory + os.sep + 'step3_btms_sizing' + os.sep + 'figures'
os.makedirs(figure_directory, exist_ok=True)

In [26]:
#import step3_btms_sizing results
sizing_results_path = os.path.join(result_parent_directory, 'step3_btms_sizing')

# load all results from path to list and add taz name as df name
sizing_results = []
for file in os.listdir(sizing_results_path):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(sizing_results_path, file), index_col=0)
        name = file.split('.')[0]
        #remove btms_sizing from name
        df.name = name.split('_')[2]
        sizing_results.append(df)

In [27]:
sizing_results[0].head(4)

,time,time_x,P_Grid,P_BTMS,E_BTMS,P_Charge,P_BTMS_Ch,P_BTMS_DCh,"param: btms size, a,b,c"
0,60.0,60,24.801494,24.801494,6.811759e-15,0.0,24.801494,-5.324878e-08,184.872065
1,120.0,120,24.801494,24.801494,3.811163e-01,0.0,24.801494,-5.324877e-08,0.657534
2,180.0,180,24.801494,24.801494,7.622326e-01,0.0,24.801494,-5.324875e-08,0.040741
3,240.0,240,24.801494,24.801494,1.143349e+00,0.0,24.801494,-5.324872e-08,0.120000


In [28]:
def stats_sizing(df):
    btms_size = df['param: btms size, a,b,c'].iloc[0]
    cost_a = df['param: btms size, a,b,c'].iloc[1] * df['E_BTMS'].max() 
    cost_b = df['param: btms size, a,b,c'].iloc[2] * df['E_BTMS'].max()
    cost_c = df['param: btms size, a,b,c'].iloc[3] * (sum(df['P_Grid'][:-2] * timestep/3600) - sum(df['P_Charge'][:-2] * timestep/3600))
    cost_total = cost_a + cost_b + cost_c
    c_rate = df['P_BTMS'].abs().max() / btms_size
    use_rate = sum(df['P_BTMS_DCh'][:-2].abs()*timestep/3600) / btms_size
    btms_ratio = sum(df['P_BTMS_DCh'][:-2].abs()*timestep/3600) / sum(df['P_Charge'][:-2]*timestep/3600)
    load_factor = df['P_Grid'][:-2].mean() / df['P_Grid'][:-2].max()

    return {'btms_size': btms_size, 'cost_a': cost_a, 'cost_b': cost_b, 'cost_c': cost_c, 'cost_total': cost_total, 'c_rate': c_rate, 'use_rate': use_rate, 'btms_ratio': btms_ratio, 'load_factor': load_factor}
    

In [29]:
# create df with stats for each taz
stats = pd.DataFrame()
for df in sizing_results:
    stats = stats.append(pd.DataFrame(stats_sizing(df), index=[df.name]))
# save stats
stats.to_csv(os.path.join(figure_directory, 'stats.csv'))

In [30]:
# plot results
from tqdm import tqdm
for df in tqdm(sizing_results):
    pltSettings()
    # use seaborn style
    plt.style.use('seaborn')
    # box around legend
    plt.rcParams['legend.frameon'] = True
    fig, ax = plt.subplots(2,1, figsize=(10,10), sharex=True, gridspec_kw={'height_ratios': [2, 1]} )
    time_in_h = df['time']/3600
    stats = stats_sizing(df)

    btms_size = df['param: btms size, a,b,c'].iloc[0]

    ax[0].step(time_in_h, df['P_BTMS'], label=r'$P_\Theta$')
    ax[0].step(time_in_h, df['P_Charge'], label=r'$P_{Charge}$')
    ax[0].step(time_in_h, df['P_Grid'], label=r'$P_{Grid}$')
    ax[0].set_title('Power usage at ' + df.name)
    ax[0].set_ylabel('Power [kW]')
    # legend outside of plot
    ax[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    ax[1].plot(time_in_h, df['E_BTMS'] - df['E_BTMS'].min(), label=r'$E_\Theta$')
    ax[1].hlines(btms_size, 0, max(time_in_h), label='BTMS-Size', linestyles='dashed')
    ax[1].set_ylabel('Energy [kWh]')
    ax[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    ax[1].set_xlabel('Time [h]')
    ax[1].set_xlim(0, max(time_in_h))

    # add text box with informations, costs and sizing results
    textstr1 = '\n'.join([
        r'$cost_{a}= \$ %.2f$' % stats['cost_a'],
        r'$cost_{b}= \$ %.2f$' % stats['cost_b'],
        r'$cost_{c}= \$ %.2f$' % stats['cost_c'],
        r'$cost_{total}= \$ %.2f$' % stats['cost_total'],])
    textstr2 = '\n'.join([
        r'$BTMS-Size =%.2f$ kWh' % stats['btms_size'],
        r'$\max{P_{\Theta}}=%.2f$ kW' % df['P_BTMS'].abs().max(),
        r'$\max{P_{Grid}}=%.2f$ kW' % df['P_Grid'].max(),
        r'C-Rate: %.2f' % stats['c_rate'],
        r'Use-Rate: %.2f' % stats['use_rate'],
        r'BTMS-Ratio: %.2f' % stats['btms_ratio'],
        r'Load-Factor(Grid): %.2f' % stats['load_factor'],
        ])
    textstr = '\n'.join([
        textstr1, 
        textstr2])
    
    # add text under legend of first plot
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax[0].text(1.05, 0.10, textstr, transform = ax[0].transAxes, fontsize = 10, bbox=props, va='bottom', ha='left')

    plt.savefig(os.path.join(figure_directory, df.name + '_btms_sizing.png'), dpi=300, bbox_inches='tight')
    plt.close()

100%|██████████| 157/157 [02:41<00:00,  1.03s/it]
